In [20]:
import os
import pandas as pd
import json
import re

In [ ]:
def walk_through_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                subreddit, date = re.match(directory+'/(.+)/(.+)', root).groups()
                json_files.append(os.path.join(root, file))
    return json_files

walk_through_json_files('scraping/data')

scraping/data/Republican/date_03-05-2025_time_17-49
[]
1ip0u0d.json


In [40]:
root = 'scraping/data/Republican/date_03-05-2025_time_17-49'
directory = 'scraping/data'
subreddit, date = re.match(directory+'/(.+)/(.+)', root).groups()
subreddit, date

('Republican', 'date_03-05-2025_time_17-49')

In [ ]:
json_paths = find_json_files('scraping/data')

for path in json_paths:
    with open(path) as file:
        comment_forest = json.load(file)
    

def process_json_file(comment_forrest):
    """
    Calculataes feature scores for a JSON file.
    """

    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Compute values (replace with actual logic)
    coalition = data.get("coalition", None)
    credibility = data.get("credibility", None)
    defection = data.get("defection", None)
    onesidedness = data.get("onesidedness", None)

    return {
        "file_path": file_path,
        "coalition": coalition,
        "credibility": credibility,
        "defection": defection,
        "onesidedness": onesidedness,
    }
